# This is the final code for the pdf insight AI

If you want to see the basic core output of the model then, you can download this file and run and see the results locally on your system using any IDE like : Jupyter notebook, VS code, or even online on google colab

if you want the full streamlit based application then download the app.py and pdf_qa_engine.py and run them locally on anyIDE

In [7]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

# Step 1: Ask user for PDF path
pdf_path = input("Enter the full path to your PDF file: ").strip()

# Step 2: Load and check number of pages
if not os.path.isfile(pdf_path):
    print("❌ File not found! Please check the path.")
    exit()

loader = PyPDFLoader(pdf_path)
all_pages = loader.load()

if len(all_pages) > 20:
    print("❌ PDF is too long! Please upload a PDF with 20 pages or fewer.")
    exit()

# Step 3: Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(all_pages)

# Step 4: Create sentence embeddings
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding_model)

# Step 5: Create retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

# Step 6: Load LLM (Flan-T5)
llm_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    tokenizer="google/flan-t5-base",
    max_length=512,
    temperature=0
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Step 7: Prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Answer the question based on the context below.
If the answer isn't in the context, say you don't know.

Context: {context}

Question: {question}

Answer:
"""
)

# Step 8: Question answering function
def ask_question(question):
    retrieved_docs = retriever.invoke(question)
    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
    final_prompt = prompt_template.format(context=context_text, question=question)
    response = llm.invoke(final_prompt)
    print("\n📌 Answer:", response)

# Step 9: User interaction loop
if __name__ == "__main__":
    print("\n✅ PDF loaded successfully! You can now ask questions about it.")
    while True:
        user_input = input("\n📝 Ask a question (or type 'exit' to quit): ")
        if user_input.lower() == "exit":
            print("👋 Exiting...")
            break
        ask_question(user_input)


Enter the full path to your PDF file: /content/drive/MyDrive/ask my pdf/defect detection in fabric.pdf


/tmp/ipython-input-1345260925.py:30: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...


📌 Answer: Iterative transfer learning involves gradually in- troducing new classes of defects to the model in a sequential manner. This strategy enables the model to adapt incrementally without forgetting previously learned classes—a challenge known as catastrophic forgetting in deep learning.

📝 Ask a question (or type 'exit' to quit): what are the defects detected in fabric


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



📌 Answer: Stains, Tear Cut, Cut, Holes and overall performance

📝 Ask a question (or type 'exit' to quit): quit


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



📌 Answer: not in the context

📝 Ask a question (or type 'exit' to quit): exit
👋 Exiting...
